In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

import seaborn as sns ##Importação para aumentar as capacidades de plot
sns.set_theme() ##Colocando o tema padrão do Seaborn
sns.set_style('whitegrid')

import missingno as msno

from sklearn.preprocessing import StandardScaler
from scipy import stats

df = pd.read_csv(filepath_or_buffer='./Nvidia_stock_data.csv')

In [6]:
print(df) #Não excluir

            Date       Close        High         Low        Open      Volume
0     1999-01-22    0.037612    0.044775    0.035581    0.040119  2714688000
1     1999-01-25    0.041552    0.042029    0.037612    0.040597   510480000
2     1999-01-26    0.038328    0.042865    0.037731    0.042029   343200000
3     1999-01-27    0.038209    0.039403    0.036298    0.038447   244368000
4     1999-01-28    0.038089    0.038447    0.037850    0.038209   227520000
...          ...         ...         ...         ...         ...         ...
6632  2025-06-04  141.919998  142.389999  139.539993  142.190002   167120800
6633  2025-06-05  139.990005  144.000000  138.830002  142.169998   231397900
6634  2025-06-06  141.720001  143.270004  141.509995  142.509995   153986200
6635  2025-06-09  142.630005  145.000000  141.940002  143.190002   185114500
6636  2025-06-10  143.960007  144.289993  141.529999  142.690002   155372400

[6637 rows x 6 columns]
